In [18]:
import pandas as pd
import numpy as np
import regression_tools

import importlib
importlib.reload(regression_tools)

pd.set_option("display.max_columns",500)

## Load Data


In [19]:
df_simd = pd.read_csv("data/simd_2020_ward.csv", index_col=0)
df_stopsearch = pd.read_csv("data/stop_search_clean.csv", index_col=0)
df_stopsearch = df_stopsearch.rename(columns={"Multi Member Ward":"ward_code"})

df_init = df_stopsearch.merge(right=df_simd, how="left", on="ward_code", validate="many_to_one")
df_init


,ward_code,Year,Month,Day of the Week,Time of the Day,Search Result,Nominal Age,Desc Drugs Found_Class B,Desc Drugs Found_Class C,Desc Drugs Found_Class A,Desc Off Weap Found_Other,Desc Other Items Found_Other,Desc Off Weap Found_Bladed Instrument,Desc Other Items Found_Firearms,Desc Off Weap Found_Pointed Article,Desc Other Items Found_Stolen Property,Desc Off Weap Found_Blunt Instrument,Desc Tobacco Found_Tobacco,Desc Other Items Found_Disposal Of Cash,Desc Alcohol Found_Beer,Desc Tobacco Found_Cigarette Papers,Desc Drugs Found_Nps,Desc Alcohol Found_Wine,Desc Alcohol Found_Spirits,Desc Other Items Found_Crossbows,Desc Other Items Found_Controlled Container,Desc Alcohol Found_Cider,Desc Other Items Found_Fireworks,Ethnic Group_2,Ethnic Group_3,Ethnic Group_4,Ethnic Group_5,Ethnic Group_6,Ethnic Group_7,Ethnic Group_8,Ethnic Group_9,Ethnic Group_10,Ethnic Group_11,Ethnic Group_12,Ethnic Group_13,Ethnic Group_14,Ethnic Group_15,Ethnic Group_16,Ethnic Group_17,Ethnic Group_18,Ethnic Group_19,Ethnic Group_20,Search Statute_CONTROLLED CONTAINERS - Sec 21 Criminal Law (Consolidation) (Scotland) Act 1995,Search Statute_CONTROLLLED ARTICLE OR SUBSTANCE - Sec 21 Criminal Law (Consolidation) (Scotland) Act 1995,Search Statute_Edinburgh Airport Byelaw 9(2) (Search in interest of public safety),"Search Statute_Sec 109(1) Copyright, Designs and Patents Act 1988 (Piracy and trademark offences etc)",Search Statute_Sec 11 of the Protection of Badgers Act 1992 (Wildlife Crime),Search Statute_Sec 11A of the Fireworks Act 2003,Search Statute_Sec 19 of the Wildlife and Countryside Act 1981,Search Statute_Sec 20 & 32 Police and Fire Reform (Scotland) Act 2012 (Other),Search Statute_Sec 20 & 32 Police and Fire Reform (Scotland) Act 2012 (Protection of Life),Search Statute_Sec 23(2) of the Misuse of Drugs Act 1971 (Drugs),Search Statute_Sec 23(3) Misuse Of Drugs Act 1971,Search Statute_Sec 23A of the Misuse of Drugs Act 1971 (Drugs),"Search Statute_Sec 24(1) Forgery and Counterfeiting Act, 1981 (Search for counterfeit money)",Search Statute_Sec 27 Deer (Scotland) Act 1996,Search Statute_Sec 289 Proceeds of Crime Act 2002 (Cash),Search Statute_Sec 4 of the Crossbows Act 1987,Search Statute_Sec 46(1) Firearms Act 1968,Search Statute_Sec 47(1) Firearms Act 1968 (Firearms),Search Statute_Sec 48(1) of the Criminal Law (Consolidation) (Scotland) Act 1995 (Offensive Weapon),Search Statute_Sec 49B of the Criminal Law (Consolidation) (Scotland) Act 1995 (Bladed Weapon/Offensive Weapon Schools),Search Statute_Sec 50 of the Criminal Law (Consolidation) (Scotland) Act 1995 (Bladed Weapon),Search Statute_Sec 53(1) of The Salmon and Freshwater Fisheries (Consolidation)(Scotland) Act 2003,Search Statute_Sec 60 Criminal Justice and Public Order Act 1994,Search Statute_Sec 60(1) Civic Government (Scotland) Act 1982 (Stolen Property),Search Statute_Sec 66 Criminal Justice (Scotland) Act 2016 (Removal of person),Search Statute_Sec 7 of the Wild Mammals (Scotland) Act 2002 (Wildlife Crime),Search Statute_Warrant - Other,Nominal Gender_Male,Nominal Physical Search Type_STRIP_IS,Nominal Physical Search Type_STRIP_OS,Command Code_2.0,Command Code_3.0,Division Code_3.0,Division Code_4.0,Division Code_5.0,Division Code_6.0,Division Code_7.0,Division Code_8.0,Division Code_9.0,Division Code_10.0,Division Code_11.0,Division Code_12.0,Division Code_13.0,Division Code_14.0,Area Command Code_3.0,Area Command Code_4.0,Area Command Code_6.0,Area Command Code_7.0,Area Command Code_8.0,Area Command Code_10.0,Area Command Code_11.0,Area Command Code_13.0,Area Command Code_14.0,Area Command Code_15.0,Area Command Code_16.0,Area Command Code_17.0,Area Command Code_18.0,Area Command Code_20.0,Area Command Code_21.0,Area Command Code_22.0,Area Command Code_24.0,Area Command Code_25.0,Area Command Code_27.0,Area Command Code_28.0,Area Command Code_29.0,Area Command Code_30.0,Area Command Code_31.0,Area Command Code_32.0,Area Command Code_34.0,Area Command Code_35.0,Area Command Code_38.

In [20]:
label = "Search Result"
s_labels = df_init[label]
df_init = df_init.drop(label, axis=1)
df_init = df_init.drop("ward_code", axis=1)

## Split Data

In [21]:
from sklearn.model_selection import train_test_split

X_train_init, X_val_init, y_train, y_val = train_test_split(df_init.copy(), s_labels.copy(), random_state=42, test_size=0.5)
X_test_init, X_val_init, y_test, y_val = train_test_split(X_val_init, y_val, random_state=42, test_size=0.5)

# Initial Model

Intuitively we might expect the stop and search data on its own to have some predictive power in the outcome of a stop and search. We'll first build this model and then try and improve upon it with the deprivation data.

We have a lot of features even in just the stop and search data - many of them one hot encodings of categorical variables. We will try and cut them down to what we think are the most useful initially.

- The date/time columns are difficult to work with, so we will drop those initially.
- Descriptions of items found are not predicting features, they would be targets and these can be our stretch goal for the project.
- The search statute and search reason could be useful but have a feeling they could be well correlated, can we explore this?
- Age, gender, ethnicity are definitely going to be important.
- Command Code, Area Command Code and Council code could be useful to link areas that are close together or operate under the same police command.

Initially, we'll start with just Age, Gender and Ethinicity


In [22]:
cols = X_train_init.columns[X_train_init.columns.str.contains("Ethnic Group.*", regex=True)].to_list()
cols += ["Nominal Age", "Nominal Gender_Male"]

# X_train, X_val = regression_tools.select_multi_cols([X_train, X_val], cols)
X_train = X_train_init[cols]
X_val = X_val_init[cols]



from sklearn.linear_model import LogisticRegression

models = {}

model="initial"
#don't apply regularisation: our features are not standaridezed and we want to avoid this to add interpretation
models[model] = LogisticRegression(
    penalty="none",
    max_iter=100
    )

models[model].fit(X_train, y_train)
print(f"Training score: {models[model].score(X_train, y_train)}\nValidation Score:{models[model].score(X_val, y_val)}")

Training score: 0.6422696484249005
Validation Score:0.6461323442455518


In [29]:
coefficients = regression_tools.get_feature_weights(cols, models[model])

#sort by absolute value
coefficients_sorted = [ (v, abs(v), k) for k,v in coefficients.items()]
coefficients_sorted.sort(reverse=True)

# print fromlargest to smallest
for val, _, key in coefficients_sorted:
    print("{: .2e}: {}".format(val, key))

 6.90e-01: Ethnic Group_10
 6.83e-01: Ethnic Group_18
 4.34e-01: Ethnic Group_12
 3.50e-01: Ethnic Group_8
 3.18e-01: Ethnic Group_19
 2.97e-01: Ethnic Group_13
 2.07e-01: Ethnic Group_5
 1.37e-01: Nominal Gender_Male
 1.03e-01: Ethnic Group_2
 5.69e-02: Ethnic Group_11
 2.99e-02: Ethnic Group_16
 1.82e-02: Ethnic Group_3
 1.05e-02: Ethnic Group_14
-1.28e-04: Nominal Age
-6.32e-02: Ethnic Group_6
-1.05e-01: Ethnic Group_9
-1.09e-01: Ethnic Group_15
-1.66e-01: Ethnic Group_7
-2.13e-01: Ethnic Group_17
-2.85e-01: Ethnic Group_20
-3.58e-01: Ethnic Group_4


The training and validation scores are both on the low side and close together, suggesting we may have high bias.

To do:
- False negative, true negative, false positive, true positive
- auc-roc -> https://www.analyticsvidhya.com/blog/2020/06/auc-roc-curve-machine-learning/
- solver 
- n_jobs for speed

In [24]:
y_train[y_train==1].count(), y_train[y_train==0].count()

(13313, 23891)

In [25]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_train, models[model].predict_proba(X_train)[:,1])

0.5203603817574762

In [26]:
#set threshold: predict_proba()

#some sort of nice graph:

# import eli5
# from eli5.sklearn import PermutationImportance

# perm = PermutationImportance(my_model, random_state=1).fit(val_X, val_y)
# eli5.show_weights(perm, feature_names = val_X.columns.tolist())
